In [ ]:
# Importing the required libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import sqlite3
from datetime import datetime

In [ ]:
#download exchange rate file
import requests

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv"
response = requests.get(url)

with open("exchange_rate.csv", "wb") as file:
    file.write(response.content)

print("File downloaded successfully.")


File downloaded successfully.


In [ ]:
er_csv = pd.read_csv('/content/exchange_rate.csv')

exchange_rate = er_csv.set_index('Currency').to_dict()['Rate']
exchange_rate

{'EUR': 0.93, 'GBP': 0.8, 'INR': 82.95}

In [ ]:
# Code for ETL operations on Country-GDP data

def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format) #format datetime 'now'
    with open("./code_log.txt","a") as f:
        f.write(timestamp + ' : ' + message + '\n')


def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    page = requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns = table_attribs)
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')
    for row in rows:
      col = row.find_all('td')
      if len(col) != 0:
        mcap = str(col[2].contents[0]) # this is market cap
        mcap = float(mcap[0:len(mcap)])
        links = col[1].find_all('a')
        # print(links[1].contents[0])
        # print(col[2].contents[0])
        data_dict = {"Name": links[1].contents[0], "MC_USD_Billion": mcap }
        df1 = pd.DataFrame(data_dict, index=[0])
        df = pd.concat([df,df1], ignore_index=True)
    df.to_csv('Largest_banks_data.csv')
    return df


def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    df['MC_GBP_Billion'] = [np.round(x*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]
    # df.to_csv('Largest_banks_data.csv')
    print(df)
    return df

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path)
    # df.to_csv('Largest_banks_data.csv')


def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)


def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''

url = 'https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ['Name', 'MC_USD_Billion']
db_name = 'World_Economies.db'
table_name = 'Largest_banks'
csv_path = '/content/Largest_banks_data.csv'
output_path = './Largest_banks_data.csv'
log_progress("Preliminaries complete. Initiating ETL process")


In [ ]:
extract(url, table_attribs)
log_progress("Data extraction complete. Initiating Transformation process")

<ipython-input-21-50b28a898d79>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,df1], ignore_index=True)


In [ ]:
df = pd.read_csv(csv_path)

transform(df, csv_path)
log_progress("Data transformation complete. Initiating Loading process")

load_to_csv(df, output_path)
log_progress("Data saved to CSV file")

   Unnamed: 0                                     Name  MC_USD_Billion  \
0           0                           JPMorgan Chase          432.92   
1           1                          Bank of America          231.52   
2           2  Industrial and Commercial Bank of China          194.56   
3           3               Agricultural Bank of China          160.68   
4           4                                HDFC Bank          157.91   
5           5                              Wells Fargo          155.87   
6           6                        HSBC Holdings PLC          148.90   
7           7                           Morgan Stanley          140.83   
8           8                  China Construction Bank          139.82   
9           9                            Bank of China          136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          402.62          346.34        35910.71  
1          215.31          185.22        19204.58  
2          180.94          15

In [ ]:
sql_connection = sqlite3.connect('Banks.db')
log_progress("SQL Connection initiated")

load_to_db(df, sql_connection, table_name)
log_progress("Data loaded to Database as a table, Executing queries")

query_statement = f"SELECT * FROM Largest_banks"
run_query(query_statement, sql_connection)

query_statement = f"SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_query(query_statement, sql_connection)

query_statement = f"SELECT Name from Largest_banks LIMIT 5"
run_query(query_statement, sql_connection)

log_progress("Process Complete")

sql_connection.close()
log_progress("Server Connection closed")

SELECT * FROM Largest_banks
   Unnamed: 0                                     Name  MC_USD_Billion  \
0           0                           JPMorgan Chase          432.92   
1           1                          Bank of America          231.52   
2           2  Industrial and Commercial Bank of China          194.56   
3           3               Agricultural Bank of China          160.68   
4           4                                HDFC Bank          157.91   
5           5                              Wells Fargo          155.87   
6           6                        HSBC Holdings PLC          148.90   
7           7                           Morgan Stanley          140.83   
8           8                  China Construction Bank          139.82   
9           9                            Bank of China          136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          402.62          346.34        35910.71  
1          215.31          185.22        19204.58  
2